# Adiabatically Assisted Variational Quantum Eigensolver [1]
In this tutorial, we are going to provide a hybrid classical-quantum algorithm to solve optimization problems in current 
quantum computers, whose basic idea is to assist variational quantum eigensolvers with adiabatic change of the Hamiltonian. We are going provide a executable example using Rigetti Forest Software Development Kit which includes pyQuil, the Rigetti Quil Compiler, and the Quantum Virtual Machine [2] and Grove Python Library [3].

### Contributors

Carlos Bravo Prieto


## Introduction
Nowadays, there is a class of quantum algorithms that are very promising for the noisy intermediate-scale quantum computing era. These algorithms are the so-called variational quantum algorithms, and are used to solve particular tasks, such as optimization. In this tutorial we present an algorithm that blend the well-known variational quantum eigensolver [4] with the adiabatic theorem. 

On one hand, the basic idea of variational quantum eigensolvers (VQE) is to consider quantum circuits parametrically characterized by a set of classical parameters. By proper tunning the circuit parameters we construct a variational ansatz to a problem under consideration. The algorithm is constructed so that the classical characterization of the quantum circuit can be subject to numerical optimization. By iteratively improving the circuit, the variational ansatz will provide a good solution.

On the other hand, we have the idea of adiabatic quantum computation. The ground state of a given Hamiltonian can be obtained by first starting from the ground state of a simple Hamiltonian $H_0$. Then the adiabatic theorem states that the two ground states can be connected by evolving with a Hamiltonian that keeps changing in time

\begin{align}
H(s) = (1-s)H_0 + s H_p \, ,
\end{align}

where the first Hamiltonian $H_0$ can be chosen to have the simple product structure of pauli operators and $H_P$ is a problem Hamiltonian whose ground state encodes the problem solution. The parameter $s$ can be
made to change in time from 0 to 1. Given this conditions, if the evolution remains slow enough, the adiabatic theorem states that the initial ground state of $H_0$ will evolve to the ground state of $H_P$ providing the solution to the problem.

We have to take into account that VQE faces the problem of finding a reasonable path in the parameter space of the circuit to end up with the correct solution. Given the exponential size of the Hilbert space, any technique that searches for paths in the parameter space that characterizes the quantum circuit is bound to deal with very tiny gradients. These gradients can even be exponentially small and no convergence to a good result would be seen [5]. 

The Adiabatically Assisted Variational Quantum Eigensolver (AAVQE) is an alternative strategy circumventing the barren plateau issue, breaking the optimization into easier, smaller parts. The interpolation parameter is used to adjust the Hamiltonian from one VQE run to the next and the state preparation parameters at each step are initialized by the
optimized parameters of the previous step.

In the following section we are going to provide a tutorial implementing the AAVQE on Forest and a executable example.

## Algorithm
Let’s show how you can run a simple AAVQE algorithm on the Quantum Virtual Machine. First we start with the relevant imports and creating the program.


In [1]:
from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import RX, RZ, CNOT, H, X, CZ, RY, SWAP, PHASE
from pyquil.paulis import sZ, sX, sY
from grove.pyvqe.vqe import VQE
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import numpy as np

qvm = QVMConnection()
p = Program()

Now we construct our ansatz pyQuil program for a 6 qubits problem. Any Python function that takes a list of numeric parameters and outputs a pyQuil program can be used as an ansatz function. In this particular example we are going to take a parameter list with 18 parameters.

In [2]:
def ansatz(params):
    """
    Quantum circuit to be initialized with a classical set of parameters.
    """
    return Program(RY(params[0], 0),
                   RY(params[1], 1),
                   RY(params[2], 2),
                   RY(params[3], 3),
                   RY(params[4], 4),
                   RY(params[5], 5),
                   CZ(0, 1), CZ(2, 3), CZ(4, 5),
                   RY(params[6], 0),
                   RY(params[7], 1),
                   RY(params[8], 2),
                   RY(params[9], 3),
                   RY(params[10], 4),
                   RY(params[11], 5),
                   CZ(1, 2), CZ(3, 4), CZ(5, 0),
                   RY(params[12], 0),
                   RY(params[13], 1),
                   RY(params[14], 2),
                   RY(params[15], 3),
                   RY(params[16], 4),
                   RY(params[17], 5))

We initialize our list of parameters from $0$ to $2\pi$ randomly. You can put particular values to the initial parameters if needed. We also initialize the number of total optimization steps and number of adiabatic steps (number of times that our Hamiltonian is going to change over the whole computation).

In [3]:
"""
Classical set of parameters, # of optimization steps and # of adiabatic steps
of the Hamiltonian.
"""

initial_angle = np.random.uniform(0.0, 2*np.pi, size = 18)
totalsteps = 5000
T = 4

Now we compute the number of optimization steps on every adiabatic step. The last adiabatic step (when the Hamiltonian is our problem Hamiltonian) will have the same number of optimization steps as the rest plus the residual steps.

In [4]:
"""
We compute the # of maximum optimization steps after every change of Hamiltonian and
the residual optimization steps. The last Hamiltonian (problem Hamiltonian) will
have the # of maximum optimization steps plus the residual steps.
"""
maxiterations = totalsteps/(T+1)
residualsteps = (maxiterations - int(maxiterations))
residualsteps = int(round((residualsteps*(T+1))))
maxiterations2 = int(maxiterations) + residualsteps

We now use the vqe module in Grove to construct an object to perform our VQE algorithm. In this example, we use scipy.optimize.minimize() with Nelder-Mead as our classical minimizer, but you can choose other parameters or write your own minimizer. We use two different vqe module with different maximum evaluations of the function, depending whether we are in intermediate adiabatic steps or in the final adiabatic step.

In [5]:
"""
Initialize with `inst = VQE(minimizer)` where `minimizer` is a function that 
performs a gradient free minization. `vqe_inst2` is used for the last optimization
steps of the problem Hamiltonian
"""
vqe_inst = VQE(minimizer=minimize,
           minimizer_kwargs={'method': 'Nelder-mead', 'tol': 1e-40, 
                             'options': {'maxiter': int(maxiterations), 
                                         'maxfev': int(maxiterations), 'disp': True}})
vqe_inst2 = VQE(minimizer=minimize,
           minimizer_kwargs={'method': 'Nelder-mead', 'tol': 1e-40, 
                             'options': {'maxiter': int(maxiterations2), 
                                         'maxfev': int(maxiterations2), 'disp': True}})

Before we run our minimizer, we have to define our 6 qubit Hamiltonians. Our easy Hamiltonian is a simple structure of sigma-Z operators. Our problem Hamiltonian is the Heisenberg Hamiltonian with periodic boundary conditions.

In [6]:
"""
Easy Hamiltonian (Hb) and Problem Hamiltonian (Hp)
"""
hamiltonianB = sum(sZ(i) for i in range(6))
hamiltonianP = (sum(sX(i)*sX(i+1) + sY(i)*sY(i+1) + sZ(i)*sZ(i+1) for i in range(5)) +
                sX(5)*sX(0) + sY(5)*sY(0) + sZ(5)*sZ(0))

Finally, we run our AAVQE, breaking the VQE optimization into smaller and easier parts. The interpolation parameter is used to adjust the Hamiltonian from one VQE run to the next and the state preparation parameters at each step are initialized by the
optimized parameters of the previous step.

In [7]:
"""
Call `vqe_inst.vqe_run` and `vqe_inst2.vqe_run`. Returns the optimal parameters, 
minimum expecation and groundstate. We adiabatically evolve the Hamiltonian as
H = (1-s)*Hb + s*Hp where 's' goes from 0 to 1.
"""
for t in range(T+1):
    s = t/T
    if s != 1:
        print('----------------------------------------------------------------------------------------------------')
        print('s = ', s)
        print('Initial parameters: ', initial_angle)
        hamiltonian = (1 - s)*hamiltonianB + s*hamiltonianP
        result = vqe_inst.vqe_run(ansatz, hamiltonian, initial_angle, None, qvm=qvm, disp = None, return_all=True)
        initial_angle = result.x
        print('Final parameters: ', result.x)
        print('Final energy: ', result.fun)

    if s == 1:
        print('----------------------------------------------------------------------------------------------------')
        print('s = ', s)
        print('Initial parameters: ', initial_angle)
        hamiltonian = (1 - s)*hamiltonianB + s*hamiltonianP
        result = vqe_inst2.vqe_run(ansatz, hamiltonian, initial_angle, None, qvm=qvm, disp = None, return_all=True)
        initial_angle = result.x
        wf = qvm.wavefunction(ansatz(result.x))
        wf = wf.amplitudes
        print('Final parameters: ', result.x)
        print('Final energy: ', result.fun)
        print('Grounstate: ', wf)

----------------------------------------------------------------------------------------------------
s =  0.0
Initial parameters:  [4.77391015 3.55158084 1.13588536 2.44123449 3.83291016 4.71735965
 5.55043762 6.05604137 5.51406634 0.8731352  6.13350531 1.49448066
 2.38998761 4.15979407 5.00221916 3.0601634  0.35635196 0.96010411]
                     models will be ineffective
energy:  -3.3032755641689726 call:  100
energy:  -4.098296955033365 call:  200
energy:  -4.798308639996513 call:  300
energy:  -5.481541739767512 call:  400
energy:  -5.742272120648856 call:  500
energy:  -5.900969826052283 call:  600
energy:  -5.942915875122804 call:  700
energy:  -5.950723968184599 call:  800
energy:  -5.953740650146127 call:  900
energy:  -5.954389432797528 call:  1000
Final parameters:  [2.63030499 3.12307949 1.26340795 3.15871191 3.15267331 6.00798652
 2.62020692 6.24355289 3.64240578 1.26738276 6.32672128 1.85949937
 3.11019636 6.25396326 5.50749071 1.26389486 0.30391184 1.01758472]
Final 

The error of the result will depend on the number of gates and parameters in our ansatz circuit. The more parameters and gates the less the error in the result [6]. The AAVQE will perform best (compared to VQE) at high depth circuits with high number of parameters.

Some results of AAVQE computations are presented. The first figure is a comparison between VQE and AAVQE performance, for the 6 qubits Ising model with 8 adiabatic steps. We can see that we can reach the convergence faster using AAVQE, using 3000 optimization steps intead of 5000.

![Ising_VQE_AAVQE](c0mparing.png)

In the following figure we show the fidelity over each optimization step for the 12 qubits Ising model with 7 adiabatic steps, using different entangling gates in our ansatz circuit. We can observe the slop downs due to the change of Hamiltonian on each adiabatic step, but rapidly increasing the fidelity over the next few optimization steps.

![Fidelity](fidelity.png)

## References

[1] A. Garcia-Saez and J.I. Latorre, "Addressing hard classical problems with 
Adiabatically Assisted Variational Quantum Eigensolvers", arXiv:1806.02287 (2018).

[2] https://pyquil.readthedocs.io/en/stable/

[3] https://grove-docs.readthedocs.io/en/latest/

[4] Alberto Peruzzo, Jarrod McClean, Peter Shadbolt, Man-Hong Yung, Xiao-Qi Zhou, Peter J. Love, Alán Aspuru-Guzik and Jeremy L. O'Brien, "A variational eigenvalue solver on a quantum processor", arXiv:1304.3061 (2013).

[5] Jarrod R. McClean, Sergio Boixo, Vadim N. Smelyanskiy, Ryan Babbush and Hartmut Neven, "Barren plateaus in quantum neural network training landscapes", arXiv:1803.11173 (2018).

[6] Christopher M. Dawson and Michael A. Nielsen, "The Solovay-Kitaev algorithm", arXiv:quant-ph/0505030 (2005).